In [1]:
%%writefile query.hql

SET hive.exec.dynamic.partition.mode=nonstrict;
SET hive.exec.max.dynamic.partitions=2000;
SET hive.exec.max.dynamic.partitions.pernode=1000;

USE stackoverflow_;

DROP TABLE if exists posts_sample_external; 

CREATE EXTERNAL TABLE posts_sample_external 
(row_id string,
post_type_id string,
year string,
month string)
ROW FORMAT 
SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe' 
WITH SERDEPROPERTIES (
    "input.regex" = ".*?(?=.*\\bId=\"(\\d+)\")(?=.*\\bPostTypeId=\"(\\d+)\")(?=.*\\bCreationDate=\"(\\d+)-(\\d+)).*"
)
LOCATION '/data/stackexchange1000/posts';

Writing query.hql


In [2]:
%%writefile -a query.hql

DROP TABLE if exists posts_sample; 

CREATE TABLE posts_sample 
(count int) 
PARTITIONED BY (year string, month string);

Appending to query.hql


In [3]:
%%writefile -a query.hql

FROM posts_sample_external
INSERT OVERWRITE TABLE posts_sample
PARTITION (year, month)
SELECT count(*) as count, year, concat(year,"-",month) as month
WHERE year IS NOT NULL
GROUP BY year, month;

Appending to query.hql


In [4]:
%%writefile final_query.hql

USE stackoverflow_;

SELECT year, month, count 
FROM (
    SELECT ROW_NUMBER() OVER(ORDER BY year, month ASC) AS row_id, year, month, count
    FROM posts_sample
) final_table
WHERE row_id=3;

Writing final_query.hql


In [5]:
! hive -S -f query.hql

In [6]:
%%bash
hive -f final_query.hql

2008	2008-10	73



Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.17 seconds
Query ID = jovyan_20180515035151_5e650e2e-9e39-46f7-bba0-e5d69047f5db
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1526345025465_0008, Tracking URL = http://13199025e80b:8088/proxy/application_1526345025465_0008/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1526345025465_0008
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2018-05-15 03:51:40,016 Stage-1 map = 0%,  reduce = 0%
2018-05-15 03:51:48,721 Stage-1 map = 100%,  